<a href="https://colab.research.google.com/github/veeraveni4c3/java/blob/main/step1_data_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

print("="*80)
print("BLOCK 1: DATA PROCESSING - TCGA MESOTHELIOMA SURVIVAL ANALYSIS")
print("="*80)

BLOCK 1: DATA PROCESSING - TCGA MESOTHELIOMA SURVIVAL ANALYSIS


In [2]:
print("\n[STEP 1] Loading datasets...")

# Load gene expression data
print("  - Loading gene expression data (TCGA-MESO_STAR.csv)...")
expression_df = pd.read_csv('TCGA-MESO_STAR.csv')
print(f"    ✓ Shape: {expression_df.shape} (genes x samples)")

# Load survival data
print("  - Loading survival data (TCGA-MESO_SURVIVAL.csv)...")
survival_df = pd.read_csv('TCGA-MESO_SURVIVAL.csv')
print(f"    ✓ Shape: {survival_df.shape}")

# Load clinical data (optional, for reference)
print("  - Loading clinical data (TCGA-MESO_CLINICAL.csv)...")
clinical_df = pd.read_csv('TCGA-MESO_CLINICAL.csv')
print(f"    ✓ Shape: {clinical_df.shape}")

print(f"\n  Initial Statistics:")
print(f"    - Total genes: {expression_df.shape[0]:,}")
print(f"    - Total samples in expression: {expression_df.shape[1]-1:,}")
print(f"    - Total samples in survival: {survival_df.shape[0]:,}")


[STEP 1] Loading datasets...
  - Loading gene expression data (TCGA-MESO_STAR.csv)...
    ✓ Shape: (60660, 88) (genes x samples)
  - Loading survival data (TCGA-MESO_SURVIVAL.csv)...
    ✓ Shape: (85, 4)
  - Loading clinical data (TCGA-MESO_CLINICAL.csv)...
    ✓ Shape: (87, 83)

  Initial Statistics:
    - Total genes: 60,660
    - Total samples in expression: 87
    - Total samples in survival: 85


In [3]:
print("\n[STEP 2] Verifying sample alignment...")

# Extract sample IDs
gene_ids = expression_df['Ensembl_ID'].values
expression_samples = expression_df.columns[1:].tolist()  # Skip Ensembl_ID column
survival_samples = survival_df['sample'].tolist()

# Find common samples
common_samples = list(set(expression_samples) & set(survival_samples))
common_samples.sort()

print(f"    - Samples in expression data: {len(expression_samples)}")
print(f"    - Samples in survival data: {len(survival_samples)}")
print(f"    - Common samples: {len(common_samples)}")

if len(common_samples) == 0:
    raise ValueError("ERROR: No common samples found between datasets!")

# Align datasets
expression_aligned = expression_df[['Ensembl_ID'] + common_samples].copy()
survival_aligned = survival_df[survival_df['sample'].isin(common_samples)].copy()
survival_aligned = survival_aligned.set_index('sample').loc[common_samples].reset_index()

print(f"    ✓ Datasets aligned successfully!")
print(f"    ✓ Final sample count: {len(common_samples)}")


[STEP 2] Verifying sample alignment...
    - Samples in expression data: 87
    - Samples in survival data: 85
    - Common samples: 85
    ✓ Datasets aligned successfully!
    ✓ Final sample count: 85


In [4]:
print("\n[STEP 3] Preparing expression matrix...")

# Create expression matrix (genes x samples)
X_raw = expression_aligned.iloc[:, 1:].values.T  # Transpose to samples x genes
gene_names = expression_aligned['Ensembl_ID'].values

print(f"    - Expression matrix shape: {X_raw.shape} (samples x genes)")
print(f"    - Data type: {X_raw.dtype}")


[STEP 3] Preparing expression matrix...
    - Expression matrix shape: (85, 60660) (samples x genes)
    - Data type: float64


In [5]:
print("\n[STEP 4] Checking for missing values...")

missing_count = np.isnan(X_raw).sum()
missing_percentage = (missing_count / X_raw.size) * 100

print(f"    - Missing values: {missing_count:,} ({missing_percentage:.4f}%)")

if missing_count > 0:
    print("    - Handling missing values: Replacing with column mean...")
    from sklearn.impute import SimpleImputer
    imputer = SimpleImputer(strategy='mean')
    X_raw = imputer.fit_transform(X_raw)
    print("    ✓ Missing values imputed")
else:
    print("    ✓ No missing values found")


[STEP 4] Checking for missing values...
    - Missing values: 0 (0.0000%)
    ✓ No missing values found


In [6]:
print("\n[STEP 5] Filtering genes with >20% zero values...")

zero_percentage = (X_raw == 0).sum(axis=0) / X_raw.shape[0] * 100
genes_to_keep_zeros = zero_percentage <= 20

print(f"    - Genes with >20% zeros: {(~genes_to_keep_zeros).sum():,}")
print(f"    - Genes remaining: {genes_to_keep_zeros.sum():,}")

X_filtered = X_raw[:, genes_to_keep_zeros]
gene_names_filtered = gene_names[genes_to_keep_zeros]

print(f"    ✓ Shape after zero filtering: {X_filtered.shape}")


[STEP 5] Filtering genes with >20% zero values...
    - Genes with >20% zeros: 35,855
    - Genes remaining: 24,805
    ✓ Shape after zero filtering: (85, 24805)


In [7]:
print("\n[STEP 6] Filtering low variance genes (bottom 25%)...")

# Calculate variance for each gene
gene_variances = np.var(X_filtered, axis=0)
variance_threshold = np.percentile(gene_variances, 25)

genes_to_keep_variance = gene_variances > variance_threshold

print(f"    - Variance threshold (25th percentile): {variance_threshold:.6f}")
print(f"    - Low variance genes removed: {(~genes_to_keep_variance).sum():,}")
print(f"    - Genes remaining: {genes_to_keep_variance.sum():,}")

X_filtered = X_filtered[:, genes_to_keep_variance]
gene_names_filtered = gene_names_filtered[genes_to_keep_variance]

print(f"    ✓ Shape after variance filtering: {X_filtered.shape}")


[STEP 6] Filtering low variance genes (bottom 25%)...
    - Variance threshold (25th percentile): 0.061004
    - Low variance genes removed: 6,202
    - Genes remaining: 18,603
    ✓ Shape after variance filtering: (85, 18603)


In [8]:
print("\n[STEP 7] Applying Z-score normalization...")

scaler = StandardScaler()
X_normalized = scaler.fit_transform(X_filtered)

print(f"    - Mean of normalized data: {X_normalized.mean():.6f}")
print(f"    - Std of normalized data: {X_normalized.std():.6f}")
print(f"    ✓ Z-score normalization completed")


[STEP 7] Applying Z-score normalization...
    - Mean of normalized data: -0.000000
    - Std of normalized data: 1.000000
    ✓ Z-score normalization completed


In [9]:
print("\n[STEP 8] Creating survival data structure for scikit-survival...")

# Extract survival information
time = survival_aligned['OS.time'].values
event = survival_aligned['OS'].values.astype(bool)

# Create structured array for scikit-survival
y_survival = np.array(
    [(e, t) for e, t in zip(event, time)],
    dtype=[('event', bool), ('time', float)]
)

print(f"    - Total samples: {len(y_survival)}")
print(f"    - Events (deaths): {event.sum()} ({event.sum()/len(event)*100:.1f}%)")
print(f"    - Censored: {(~event).sum()} ({(~event).sum()/len(event)*100:.1f}%)")
print(f"    - Median survival time: {np.median(time):.1f} days")
print(f"    - Min survival time: {time.min():.1f} days")
print(f"    - Max survival time: {time.max():.1f} days")
print(f"    ✓ Survival structure created")


[STEP 8] Creating survival data structure for scikit-survival...
    - Total samples: 85
    - Events (deaths): 73 (85.9%)
    - Censored: 12 (14.1%)
    - Median survival time: 527.0 days
    - Min survival time: 20.0 days
    - Max survival time: 2790.0 days
    ✓ Survival structure created


In [10]:



print("\n[STEP 9] Creating final processed data structures...")

# Create X dataframe (samples x genes)
X_final = pd.DataFrame(
    X_normalized,
    index=common_samples,
    columns=gene_names_filtered
)

# Create y dataframe (survival info)
y_final = pd.DataFrame({
    'sample': common_samples,
    'event': event,
    'time': time
})

print(f"    ✓ X shape: {X_final.shape} (samples x genes)")
print(f"    ✓ y shape: {y_final.shape}")


[STEP 9] Creating final processed data structures...
    ✓ X shape: (85, 18603) (samples x genes)
    ✓ y shape: (85, 3)


In [11]:
print("\n[STEP 10] Generating summary statistics...")

summary_stats = {
    'Metric': [
        'Initial genes',
        'Genes after zero filtering (>20%)',
        'Genes after variance filtering (bottom 25%)',
        'Final genes',
        'Total samples',
        'Events (deaths)',
        'Censored samples',
        'Event rate (%)',
        'Median survival (days)',
        'Mean survival (days)',
        'Data reduction (%)'
    ],
    'Value': [
        f"{len(gene_names):,}",
        f"{genes_to_keep_zeros.sum():,}",
        f"{X_filtered.shape[1]:,}",
        f"{X_final.shape[1]:,}",
        f"{X_final.shape[0]}",
        f"{event.sum()}",
        f"{(~event).sum()}",
        f"{event.sum()/len(event)*100:.1f}",
        f"{np.median(time):.1f}",
        f"{np.mean(time):.1f}",
        f"{(1 - X_final.shape[1]/len(gene_names))*100:.1f}"
    ]
}

summary_df = pd.DataFrame(summary_stats)
print("\n" + summary_df.to_string(index=False))


[STEP 10] Generating summary statistics...

                                     Metric  Value
                              Initial genes 60,660
          Genes after zero filtering (>20%) 24,805
Genes after variance filtering (bottom 25%) 18,603
                                Final genes 18,603
                              Total samples     85
                            Events (deaths)     73
                           Censored samples     12
                             Event rate (%)   85.9
                     Median survival (days)  527.0
                       Mean survival (days)  668.6
                         Data reduction (%)   69.3


In [12]:
import os

print("\n[STEP 11] Saving processed data...")

# Define the output directory
output_dir = 'results/01_processed_data'

# Create the output directory if it doesn't exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    print(f"    ✓ Created output directory: {output_dir}")

# Note: Excel has max 16,384 columns, but we have 18,603 genes
# Save X as CSV, y and summary to Excel
print(f"    - Note: X has {X_final.shape[1]:,} columns (Excel max is 16,384)")
print(f"    - Saving X to CSV format instead...")

# Save X as CSV (main format)
X_final.to_csv(f'{output_dir}/X_expression.csv')
print(f"    ✓ Saved X (expression) as CSV: X_expression.csv")

# Save y and summary to Excel
output_excel = f'{output_dir}/survival_and_summary.xlsx'
print(f"    - Saving survival data and summary to: {output_excel}")

with pd.ExcelWriter(output_excel, engine='openpyxl') as writer:
    y_final.to_excel(writer, sheet_name='y_survival', index=False)
    summary_df.to_excel(writer, sheet_name='summary', index=False)

    # Add first 100 genes as a preview in Excel
    X_preview = X_final.iloc[:, :100].copy()
    X_preview.to_excel(writer, sheet_name='X_preview_100genes', index=True)

print(f"    ✓ Saved y (survival) to sheet: 'y_survival'")
print(f"    ✓ Saved summary to sheet: 'summary'")
print(f"    ✓ Saved X preview (first 100 genes) to sheet: 'X_preview_100genes'")

# Save y also as CSV for convenience
y_final.to_csv(f'{output_dir}/y_survival.csv', index=False)

print(f"    ✓ Also saved y_survival.csv")

# Save gene list
gene_list_df = pd.DataFrame({
    'gene_id': gene_names_filtered,
    'gene_index': range(len(gene_names_filtered))
})
gene_list_df.to_csv(f'{output_dir}/gene_list.csv', index=False)
print(f"    ✓ Saved gene list (n={len(gene_names_filtered):,})")

# Save sample list
sample_list_df = pd.DataFrame({
    'sample_id': common_samples,
    'sample_index': range(len(common_samples))
})
sample_list_df.to_csv(f'{output_dir}/sample_list.csv', index=False)
print(f"    ✓ Saved sample list (n={len(common_samples)})")

# Save processing log
log_text = f"""
DATA PROCESSING LOG - TCGA MESOTHELIOMA
========================================

Processing Date: {pd.Timestamp.now()}

INPUT FILES:
- Gene Expression: TCGA-MESO_STAR.csv
- Survival Data: TCGA-MESO_SURVIVAL.csv
- Clinical Data: TCGA-MESO_CLINICAL.csv

PROCESSING STEPS:
1. Loaded {len(gene_names):,} genes across {len(expression_samples)} samples
2. Aligned {len(common_samples)} common samples between datasets
3. Removed {missing_count:,} missing values ({missing_percentage:.4f}%)
4. Filtered {(~genes_to_keep_zeros).sum():,} genes with >20% zeros
5. Filtered {(~genes_to_keep_variance).sum():,} low variance genes (bottom 25%)
6. Applied Z-score normalization (mean=0, std=1)
7. Created survival structured array for scikit-survival

FINAL OUTPUT:
- Expression matrix (X): {X_final.shape[0]} samples × {X_final.shape[1]:,} genes
- Survival data (y): {y_final.shape[0]} samples with {event.sum()} events ({event.sum()/len(event)*100:.1f}%)
- Data reduction: {(1 - X_final.shape[1]/len(gene_names))*100:.1f}% of genes removed

SURVIVAL STATISTICS:
- Events (deaths): {event.sum()} ({event.sum()/len(event)*100:.1f}%)
- Censored: {(~event).sum()} ({(~event).sum()/len(event)*100:.1f}%)
- Median survival: {np.median(time):.1f} days
- Mean survival: {np.mean(time):.1f} days
- Range: {time.min():.1f} - {time.max():.1f} days

FILES GENERATED:
- X_expression.csv (full expression matrix - too large for Excel)
- survival_and_summary.xlsx (3 sheets: y_survival, summary, X_preview_100genes)
- y_survival.csv
- gene_list.csv
- sample_list.csv
- processing_log.txt

STATUS: ✓ COMPLETED SUCCESSFULLY
"""

with open(f'{output_dir}/processing_log.txt', 'w', encoding='utf-8') as f:
    f.write(log_text)

print(f"    ✓ Saved processing log")


[STEP 11] Saving processed data...
    - Note: X has 18,603 columns (Excel max is 16,384)
    - Saving X to CSV format instead...
    ✓ Saved X (expression) as CSV: X_expression.csv
    - Saving survival data and summary to: results/01_processed_data/survival_and_summary.xlsx
    ✓ Saved y (survival) to sheet: 'y_survival'
    ✓ Saved summary to sheet: 'summary'
    ✓ Saved X preview (first 100 genes) to sheet: 'X_preview_100genes'
    ✓ Also saved y_survival.csv
    ✓ Saved gene list (n=18,603)
    ✓ Saved sample list (n=85)
    ✓ Saved processing log


In [13]:
print("\n" + "="*80)
print("DATA PROCESSING COMPLETED SUCCESSFULLY!")
print("="*80)
print(f"\n✓ Processed {X_final.shape[1]:,} genes across {X_final.shape[0]} samples")
print(f"✓ {event.sum()} events ({event.sum()/len(event)*100:.1f}% event rate)")
print(f"✓ All files saved to: results/01_processed_data/")
print(f"\nReady for BLOCK 2: Feature Selection")
print("="*80)


DATA PROCESSING COMPLETED SUCCESSFULLY!

✓ Processed 18,603 genes across 85 samples
✓ 73 events (85.9% event rate)
✓ All files saved to: results/01_processed_data/

Ready for BLOCK 2: Feature Selection


In [14]:
import pandas as pd
import numpy as np
from sksurv.linear_model import CoxPHSurvivalAnalysis
from statsmodels.stats.multitest import multipletests
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

print("="*80)
print("BLOCK 2: FEATURE SELECTION - TCGA MESOTHELIOMA SURVIVAL ANALYSIS")
print("="*80)

BLOCK 2: FEATURE SELECTION - TCGA MESOTHELIOMA SURVIVAL ANALYSIS


In [15]:
print("\n[STEP 1] Loading processed data from Block 1...")

# Load expression data
X_df = pd.read_csv('results/01_processed_data/X_expression.csv', index_col=0)
print(f"  [OK] Loaded X: {X_df.shape} (samples x genes)")

# Load survival data
y_df = pd.read_csv('results/01_processed_data/y_survival.csv')
print(f"  [OK] Loaded y: {y_df.shape}")

# Load gene list
gene_list = pd.read_csv('results/01_processed_data/gene_list.csv')
print(f"  [OK] Loaded gene list: {len(gene_list)} genes")

# Create survival structured array
y = np.array(
    [(bool(event), float(time)) for event, time in zip(y_df['event'], y_df['time'])],
    dtype=[('event', bool), ('time', float)]
)

print(f"\n  Data Summary:")
print(f"    - Samples: {X_df.shape[0]}")
print(f"    - Genes: {X_df.shape[1]:,}")
print(f"    - Events: {y['event'].sum()} ({y['event'].sum()/len(y)*100:.1f}%)")


[STEP 1] Loading processed data from Block 1...
  [OK] Loaded X: (85, 18603) (samples x genes)
  [OK] Loaded y: (85, 3)
  [OK] Loaded gene list: 18603 genes

  Data Summary:
    - Samples: 85
    - Genes: 18,603
    - Events: 73 (85.9%)


In [16]:
print("\n[STEP 2] Running univariate Cox regression for all genes...")
print(f"  Testing {X_df.shape[1]:,} genes individually...")
print("  This will take approximately 10-15 minutes...")

results = []
total_genes = X_df.shape[1]

for idx, gene in enumerate(X_df.columns):
    if (idx + 1) % 1000 == 0:
        print(f"  Progress: {idx+1:,}/{total_genes:,} genes ({(idx+1)/total_genes*100:.1f}%)")

    try:
        # Extract gene expression for this gene
        X_gene = X_df[[gene]].values

        # Fit Cox model
        cox = CoxPHSurvivalAnalysis()
        cox.fit(X_gene, y)

        # Get coefficient
        coef = cox.coef_[0]

        # Calculate hazard ratio
        hr = np.exp(coef)

        # Get score (for p-value calculation)
        score = cox.score(X_gene, y)

        # Calculate p-value using Wald test
        # Standard error approximation
        n_events = y['event'].sum()
        se = np.sqrt(1 / n_events)  # Simplified SE
        z_score = coef / se
        p_value = 2 * (1 - stats.norm.cdf(abs(z_score)))

        # Calculate 95% CI for HR
        hr_lower = np.exp(coef - 1.96 * se)
        hr_upper = np.exp(coef + 1.96 * se)

        results.append({
            'gene_id': gene,
            'coefficient': coef,
            'hazard_ratio': hr,
            'hr_lower_95': hr_lower,
            'hr_upper_95': hr_upper,
            'p_value': p_value,
            'concordance_index': score
        })

    except Exception as e:
        # If Cox regression fails for a gene, skip it
        results.append({
            'gene_id': gene,
            'coefficient': np.nan,
            'hazard_ratio': np.nan,
            'hr_lower_95': np.nan,
            'hr_upper_95': np.nan,
            'p_value': 1.0,
            'concordance_index': 0.5
        })

print(f"  [OK] Completed Cox regression for {len(results):,} genes")

# Create results dataframe
results_df = pd.DataFrame(results)

# Remove genes with failed regression
results_df = results_df[results_df['p_value'].notna()].copy()
print(f"  [OK] Valid results: {len(results_df):,} genes")



[STEP 2] Running univariate Cox regression for all genes...
  Testing 18,603 genes individually...
  This will take approximately 10-15 minutes...
  Progress: 1,000/18,603 genes (5.4%)
  Progress: 2,000/18,603 genes (10.8%)
  Progress: 3,000/18,603 genes (16.1%)
  Progress: 4,000/18,603 genes (21.5%)
  Progress: 5,000/18,603 genes (26.9%)
  Progress: 6,000/18,603 genes (32.3%)
  Progress: 7,000/18,603 genes (37.6%)
  Progress: 8,000/18,603 genes (43.0%)
  Progress: 9,000/18,603 genes (48.4%)
  Progress: 10,000/18,603 genes (53.8%)
  Progress: 11,000/18,603 genes (59.1%)
  Progress: 12,000/18,603 genes (64.5%)
  Progress: 13,000/18,603 genes (69.9%)
  Progress: 14,000/18,603 genes (75.3%)
  Progress: 15,000/18,603 genes (80.6%)
  Progress: 16,000/18,603 genes (86.0%)
  Progress: 17,000/18,603 genes (91.4%)
  Progress: 18,000/18,603 genes (96.8%)
  [OK] Completed Cox regression for 18,603 genes
  [OK] Valid results: 18,603 genes


In [17]:
print("\n[STEP 3] Stage 1 Filtering: FDR correction (FDR < 0.05)...")

# Apply FDR correction (Benjamini-Hochberg)
reject, pvals_corrected, alphacSidak, alphacBonf = multipletests(
    results_df['p_value'],
    alpha=0.05,
    method='fdr_bh'
)

results_df['fdr_adjusted_p'] = pvals_corrected
results_df['fdr_significant'] = reject

# Filter by FDR < 0.05
stage1_df = results_df[results_df['fdr_adjusted_p'] < 0.05].copy()

print(f"  Initial genes: {len(results_df):,}")
print(f"  Genes passing FDR < 0.05: {len(stage1_df):,}")
print(f"  Reduction: {(1 - len(stage1_df)/len(results_df))*100:.1f}%")

if len(stage1_df) > 0:
    print(f"\n  FDR Statistics:")
    print(f"    - Min p-value: {stage1_df['p_value'].min():.2e}")
    print(f"    - Max p-value: {stage1_df['p_value'].max():.2e}")
    print(f"    - Min FDR: {stage1_df['fdr_adjusted_p'].min():.2e}")
    print(f"    - Max FDR: {stage1_df['fdr_adjusted_p'].max():.2e}")


[STEP 3] Stage 1 Filtering: FDR correction (FDR < 0.05)...
  Initial genes: 18,603
  Genes passing FDR < 0.05: 3,062
  Reduction: 83.5%

  FDR Statistics:
    - Min p-value: 0.00e+00
    - Max p-value: 8.22e-03
    - Min FDR: 0.00e+00
    - Max FDR: 5.00e-02


In [18]:
print("\n[STEP 4] Stage 2 Filtering: Stricter criteria...")

# Apply stricter filters
stage2_df = stage1_df[
    (stage1_df['fdr_adjusted_p'] < 0.01) &  # Stricter p-value
    ((stage1_df['hazard_ratio'] > 1.5) | (stage1_df['hazard_ratio'] < 0.67))  # Strong effect
].copy()

print(f"  Criteria:")
print(f"    - FDR adjusted p-value < 0.01")
print(f"    - Hazard Ratio > 1.5 OR < 0.67")
print(f"\n  Genes from Stage 1: {len(stage1_df):,}")
print(f"  Genes passing Stage 2: {len(stage2_df):,}")
print(f"  Reduction: {(1 - len(stage2_df)/len(stage1_df))*100:.1f}%")

if len(stage2_df) > 0:
    risk_genes = (stage2_df['hazard_ratio'] > 1.5).sum()
    protective_genes = (stage2_df['hazard_ratio'] < 0.67).sum()
    print(f"\n  Gene Categories:")
    print(f"    - Risk genes (HR > 1.5): {risk_genes}")
    print(f"    - Protective genes (HR < 0.67): {protective_genes}")


[STEP 4] Stage 2 Filtering: Stricter criteria...
  Criteria:
    - FDR adjusted p-value < 0.01
    - Hazard Ratio > 1.5 OR < 0.67

  Genes from Stage 1: 3,062
  Genes passing Stage 2: 1,416
  Reduction: 53.8%

  Gene Categories:
    - Risk genes (HR > 1.5): 783
    - Protective genes (HR < 0.67): 633


In [19]:
print("\n[STEP 5] Stage 3 Filtering: Select top 20 genes...")

# Calculate combined score: -log10(p_value) × |log2(HR)|
stage2_df['log10_p'] = -np.log10(stage2_df['fdr_adjusted_p'])
stage2_df['log2_hr'] = np.log2(stage2_df['hazard_ratio'])
stage2_df['abs_log2_hr'] = np.abs(stage2_df['log2_hr'])
stage2_df['combined_score'] = stage2_df['log10_p'] * stage2_df['abs_log2_hr']

# Sort by combined score
stage2_df_sorted = stage2_df.sort_values('combined_score', ascending=False).copy()

# Select top 20
n_select = min(20, len(stage2_df_sorted))
top20_df = stage2_df_sorted.head(n_select).copy()

print(f"  Combined Score Formula: -log10(FDR) × |log2(HR)|")
print(f"  Genes from Stage 2: {len(stage2_df):,}")
print(f"  Top genes selected: {n_select}")

if len(top20_df) > 0:
    print(f"\n  Top 20 Gene Statistics:")
    print(f"    - Min combined score: {top20_df['combined_score'].min():.2f}")
    print(f"    - Max combined score: {top20_df['combined_score'].max():.2f}")
    print(f"    - Min HR: {top20_df['hazard_ratio'].min():.2f}")
    print(f"    - Max HR: {top20_df['hazard_ratio'].max():.2f}")

    risk_top = (top20_df['hazard_ratio'] > 1).sum()
    protective_top = (top20_df['hazard_ratio'] < 1).sum()
    print(f"    - Risk genes: {risk_top}")
    print(f"    - Protective genes: {protective_top}")

# Add gene rank
top20_df['rank'] = range(1, len(top20_df) + 1)

# Classify genes
top20_df['gene_type'] = top20_df['hazard_ratio'].apply(
    lambda x: 'Risk' if x > 1 else 'Protective'
)



[STEP 5] Stage 3 Filtering: Select top 20 genes...
  Combined Score Formula: -log10(FDR) × |log2(HR)|
  Genes from Stage 2: 1,416
  Top genes selected: 20

  Top 20 Gene Statistics:
    - Min combined score: 9.98
    - Max combined score: inf
    - Min HR: 0.37
    - Max HR: 2.77
    - Risk genes: 17
    - Protective genes: 3


In [20]:
print("\n[STEP 6] Generating summary statistics...")

summary_stats = pd.DataFrame({
    'Stage': [
        'Initial (All genes)',
        'After Cox Regression',
        'Stage 1: FDR < 0.05',
        'Stage 2: FDR < 0.01, |HR| > 1.5',
        'Stage 3: Top 20 Selected'
    ],
    'Number of Genes': [
        X_df.shape[1],
        len(results_df),
        len(stage1_df),
        len(stage2_df),
        len(top20_df)
    ],
    'Percentage': [
        '100.0%',
        f'{len(results_df)/X_df.shape[1]*100:.1f}%',
        f'{len(stage1_df)/X_df.shape[1]*100:.1f}%',
        f'{len(stage2_df)/X_df.shape[1]*100:.1f}%',
        f'{len(top20_df)/X_df.shape[1]*100:.3f}%'
    ]
})

print("\n" + summary_stats.to_string(index=False))


[STEP 6] Generating summary statistics...

                          Stage  Number of Genes Percentage
            Initial (All genes)            18603     100.0%
           After Cox Regression            18603     100.0%
            Stage 1: FDR < 0.05             3062      16.5%
Stage 2: FDR < 0.01, |HR| > 1.5             1416       7.6%
       Stage 3: Top 20 Selected               20     0.108%


In [21]:
print("\n[STEP 7] Saving results...")

# Create output directory
import os
os.makedirs('results/02_selected_features', exist_ok=True)

# Save all results
results_df_sorted = results_df.sort_values('p_value')
results_df_sorted.to_csv('results/02_selected_features/all_genes_cox_results.csv', index=False)
print(f"  [OK] Saved: all_genes_cox_results.csv ({len(results_df):,} genes)")

# Save Stage 1 results
if len(stage1_df) > 0:
    stage1_df_sorted = stage1_df.sort_values('fdr_adjusted_p')
    stage1_df_sorted.to_csv('results/02_selected_features/stage1_fdr005.csv', index=False)
    print(f"  [OK] Saved: stage1_fdr005.csv ({len(stage1_df):,} genes)")

# Save Stage 2 results
if len(stage2_df) > 0:
    stage2_df_sorted = stage2_df.sort_values('combined_score', ascending=False)
    stage2_df_sorted.to_csv('results/02_selected_features/stage2_strict.csv', index=False)
    print(f"  [OK] Saved: stage2_strict.csv ({len(stage2_df):,} genes)")

# Save Top 20
if len(top20_df) > 0:
    top20_df.to_csv('results/02_selected_features/top20_genes_for_validation.csv', index=False)
    print(f"  [OK] Saved: top20_genes_for_validation.csv ({len(top20_df)} genes)")

# Save to Excel
with pd.ExcelWriter('results/02_selected_features/feature_selection_summary.xlsx', engine='openpyxl') as writer:
    if len(top20_df) > 0:
        top20_df.to_excel(writer, sheet_name='Top20_Genes', index=False)
    if len(stage2_df) > 0:
        stage2_df_sorted.to_excel(writer, sheet_name='Stage2_Strict', index=False)
    if len(stage1_df) > 0:
        stage1_df_sorted.head(200).to_excel(writer, sheet_name='Stage1_FDR005_top200', index=False)
    summary_stats.to_excel(writer, sheet_name='Summary', index=False)

print(f"  [OK] Saved: feature_selection_summary.xlsx")



[STEP 7] Saving results...
  [OK] Saved: all_genes_cox_results.csv (18,603 genes)
  [OK] Saved: stage1_fdr005.csv (3,062 genes)
  [OK] Saved: stage2_strict.csv (1,416 genes)
  [OK] Saved: top20_genes_for_validation.csv (20 genes)
  [OK] Saved: feature_selection_summary.xlsx


In [23]:
print("\n[STEP 8] Generating visualizations...")

# Set style
sns.set_style("whitegrid")
plt.rcParams['figure.dpi'] = 300

# 1. Volcano Plot
print("  Creating volcano plot...")
fig, ax = plt.subplots(figsize=(12, 8))

# Plot all genes
ax.scatter(
    np.log2(results_df['hazard_ratio']), # Corrected line: directly calculate log2(hazard_ratio)
    -np.log10(results_df['p_value']),
    alpha=0.3,
    s=10,
    color='gray',
    label='Not significant'
)

# Highlight Stage 1 genes
if len(stage1_df) > 0:
    ax.scatter(
        np.log2(stage1_df['hazard_ratio']),
        -np.log10(stage1_df['p_value']),
        alpha=0.5,
        s=20,
        color='lightblue',
        label=f'FDR < 0.05 (n={len(stage1_df)})'
    )

# Highlight Top 20 genes
if len(top20_df) > 0:
    colors = ['red' if hr > 1 else 'green' for hr in top20_df['hazard_ratio']]
    ax.scatter(
        np.log2(top20_df['hazard_ratio']),
        -np.log10(top20_df['p_value']),
        alpha=0.8,
        s=100,
        c=colors,
        edgecolors='black',
        linewidth=1,
        label=f'Top 20 (Risk=red, Protective=green)',
        zorder=5
    )

# Add threshold lines
ax.axhline(y=-np.log10(0.05), color='blue', linestyle='--', linewidth=1, alpha=0.5, label='p = 0.05')
ax.axvline(x=np.log2(1.5), color='orange', linestyle='--', linewidth=1, alpha=0.5)
ax.axvline(x=np.log2(0.67), color='orange', linestyle='--', linewidth=1, alpha=0.5, label='HR = 1.5 / 0.67')

ax.set_xlabel('log2(Hazard Ratio)', fontsize=12, fontweight='bold')
ax.set_ylabel('-log10(P-value)', fontsize=12, fontweight='bold')
ax.set_title('Volcano Plot: Gene Expression vs Survival\nTCGA Mesothelioma Cohort',
             fontsize=14, fontweight='bold')
ax.legend(loc='upper right', framealpha=0.9)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('results/02_selected_features/volcano_plot.png', dpi=300, bbox_inches='tight')
plt.close()
print("  ✓ Saved: volcano_plot.png")

# 2. Top 20 Genes Bar Plot
if len(top20_df) > 0:
    print("  Creating top 20 genes bar plot...")
    fig, ax = plt.subplots(figsize=(10, 8))

    # Sort by HR
    top20_plot = top20_df.sort_values('hazard_ratio', ascending=True)

    # Create colors
    colors = ['green' if hr < 1 else 'red' for hr in top20_plot['hazard_ratio']]

    # Plot
    y_pos = np.arange(len(top20_plot))
    ax.barh(y_pos, top20_plot['hazard_ratio'], color=colors, alpha=0.7)
    ax.axvline(x=1, color='black', linestyle='--', linewidth=2)

    # Labels
    gene_labels = [g.split('.')[0][-10:] for g in top20_plot['gene_id']]  # Shorten gene names
    ax.set_yticks(y_pos)
    ax.set_yticklabels(gene_labels, fontsize=8)
    ax.set_xlabel('Hazard Ratio', fontsize=12, fontweight='bold')
    ax.set_title('Top 20 Genes: Hazard Ratios\nGreen=Protective, Red=Risk',
                 fontsize=14, fontweight='bold')
    ax.grid(True, alpha=0.3, axis='x')

    plt.tight_layout()
    plt.savefig('results/02_selected_features/top20_genes_HR_plot.png', dpi=300, bbox_inches='tight')
    plt.close()
    print("  ✓ Saved: top20_genes_HR_plot.png")

# 3. Selection Flowchart (text-based)
print("  Creating selection flowchart...")
fig, ax = plt.subplots(figsize=(10, 8))
ax.axis('off')

stages_text = f"""
FEATURE SELECTION FLOWCHART
═══════════════════════════════════════════════

Initial Dataset
    ↓
{X_df.shape[1]:,} genes (all)
    ↓
Univariate Cox Regression
    ↓
{len(results_df):,} genes (valid results)
    ↓
STAGE 1: FDR Correction (FDR < 0.05)
    ↓
{len(stage1_df):,} genes
    ↓
STAGE 2: Strict Filtering
    • FDR < 0.01
    • HR > 1.5 or HR < 0.67
    ↓
{len(stage2_df):,} genes
    ↓
STAGE 3: Combined Score Ranking
    • Score = -log10(FDR) × |log2(HR)|
    ↓
{len(top20_df)} genes (TOP 20)
═══════════════════════════════════════════════

Data Reduction: {(1 - len(top20_df)/X_df.shape[1])*100:.2f}%
"""

ax.text(0.5, 0.5, stages_text,
        fontsize=11,
        family='monospace',
        ha='center',
        va='center',
        bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

plt.savefig('results/02_selected_features/selection_flowchart.png', dpi=300, bbox_inches='tight')
plt.close()
print("  ✓ Saved: selection_flowchart.png")


[STEP 8] Generating visualizations...
  Creating volcano plot...
  ✓ Saved: volcano_plot.png
  Creating top 20 genes bar plot...
  ✓ Saved: top20_genes_HR_plot.png
  Creating selection flowchart...
  ✓ Saved: selection_flowchart.png


In [24]:
print("\n[STEP 9] Generating processing log...")

log_text = f"""
FEATURE SELECTION LOG - TCGA MESOTHELIOMA
==========================================

Processing Date: {pd.Timestamp.now()}

INPUT DATA:
-----------
- Expression matrix: {X_df.shape[0]} samples × {X_df.shape[1]:,} genes
- Survival data: {len(y)} samples ({y['event'].sum()} events, {(~y['event']).sum()} censored)
- Event rate: {y['event'].sum()/len(y)*100:.1f}%

UNIVARIATE COX REGRESSION:
---------------------------
- Total genes tested: {X_df.shape[1]:,}
- Valid results: {len(results_df):,}
- Failed regressions: {X_df.shape[1] - len(results_df)}

STAGE 1: FDR CORRECTION (FDR < 0.05)
-------------------------------------
- Genes passing: {len(stage1_df):,}
- Reduction: {(1 - len(stage1_df)/len(results_df))*100:.1f}%
- FDR range: {stage1_df['fdr_adjusted_p'].min():.2e} - {stage1_df['fdr_adjusted_p'].max():.2e}

STAGE 2: STRICT FILTERING
--------------------------
Criteria: FDR < 0.01 AND (HR > 1.5 OR HR < 0.67)
- Genes passing: {len(stage2_df):,}
- Risk genes (HR > 1.5): {(stage2_df['hazard_ratio'] > 1.5).sum()}
- Protective genes (HR < 0.67): {(stage2_df['hazard_ratio'] < 0.67).sum()}

STAGE 3: TOP 20 SELECTION
--------------------------
Combined Score = -log10(FDR) × |log2(HR)|
- Top genes selected: {len(top20_df)}
- Risk genes: {(top20_df['hazard_ratio'] > 1).sum()}
- Protective genes: {(top20_df['hazard_ratio'] < 1).sum()}
- Score range: {top20_df['combined_score'].min():.2f} - {top20_df['combined_score'].max():.2f}
- HR range: {top20_df['hazard_ratio'].min():.2f} - {top20_df['hazard_ratio'].max():.2f}

OVERALL STATISTICS:
-------------------
- Initial genes: {X_df.shape[1]:,}
- Final genes: {len(top20_df)}
- Data reduction: {(1 - len(top20_df)/X_df.shape[1])*100:.2f}%
- Selection rate: {len(top20_df)/X_df.shape[1]*100:.3f}%

FILES GENERATED:
----------------
- all_genes_cox_results.csv ({len(results_df):,} genes)
- stage1_fdr005.csv ({len(stage1_df):,} genes)
- stage2_strict.csv ({len(stage2_df):,} genes)
- top20_genes_for_validation.csv ({len(top20_df)} genes)
- feature_selection_summary.xlsx (4 sheets)
- volcano_plot.png
- top20_genes_HR_plot.png
- selection_flowchart.png

STATUS: COMPLETED SUCCESSFULLY

READY FOR BLOCK 3: Individual Gene KM Plots ({len(top20_df)} genes)
"""

with open('results/02_selected_features/block2_processing_log.txt', 'w', encoding='utf-8') as f:
    f.write(log_text)

print("  ✓ Saved: block2_processing_log.txt")


[STEP 9] Generating processing log...
  ✓ Saved: block2_processing_log.txt


In [25]:
print("\n" + "="*80)
print("BLOCK 2: FEATURE SELECTION COMPLETED SUCCESSFULLY!")
print("="*80)

print(f"\n[SELECTION SUMMARY]")
print(f"   Initial genes:        {X_df.shape[1]:,}")
print(f"   Stage 1 (FDR<0.05):   {len(stage1_df):,}")
print(f"   Stage 2 (Strict):     {len(stage2_df):,}")
print(f"   Final (Top 20):       {len(top20_df)}")
print(f"   Data reduction:       {(1 - len(top20_df)/X_df.shape[1])*100:.2f}%")

if len(top20_df) > 0:
    print(f"\n[TOP 20 GENES SELECTED]")
    print(f"   Risk genes (HR>1):        {(top20_df['hazard_ratio'] > 1).sum()}")
    print(f"   Protective genes (HR<1):  {(top20_df['hazard_ratio'] < 1).sum()}")
    print(f"   HR range:                 {top20_df['hazard_ratio'].min():.2f} - {top20_df['hazard_ratio'].max():.2f}")
    print(f"   Best p-value:             {top20_df['p_value'].min():.2e}")

print(f"\n[FILES] All results saved to: results/02_selected_features/")
print(f"\n[READY] For BLOCK 3: Individual Gene KM Plots")
print("="*80)

# Display top 10 genes
if len(top20_df) > 0:
    print("\n[TOP 10 GENES PREVIEW]")
    print("="*80)
    preview_df = top20_df[['rank', 'gene_id', 'p_value', 'fdr_adjusted_p',
                            'hazard_ratio', 'combined_score', 'gene_type']].head(10)
    preview_df_display = preview_df.copy()
    preview_df_display['gene_id'] = preview_df_display['gene_id'].str[:20]  # Shorten for display
    print(preview_df_display.to_string(index=False))
    print("="*80)




BLOCK 2: FEATURE SELECTION COMPLETED SUCCESSFULLY!

[SELECTION SUMMARY]
   Initial genes:        18,603
   Stage 1 (FDR<0.05):   3,062
   Stage 2 (Strict):     1,416
   Final (Top 20):       20
   Data reduction:       99.89%

[TOP 20 GENES SELECTED]
   Risk genes (HR>1):        17
   Protective genes (HR<1):  3
   HR range:                 0.37 - 2.77
   Best p-value:             0.00e+00

[FILES] All results saved to: results/02_selected_features/

[READY] For BLOCK 3: Individual Gene KM Plots

[TOP 10 GENES PREVIEW]
 rank            gene_id      p_value  fdr_adjusted_p  hazard_ratio  combined_score  gene_type
    1  ENSG00000285517.1 0.000000e+00    0.000000e+00      2.773552             inf       Risk
    2  ENSG00000260886.2 0.000000e+00    0.000000e+00      0.368709             inf Protective
    3 ENSG00000176619.13 0.000000e+00    0.000000e+00      2.664084             inf       Risk
    4  ENSG00000276043.5 0.000000e+00    0.000000e+00      2.732539             inf       Risk